In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = f"https://www.uniprot.org/uniprot/O14786"
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.text, 'lxml')

In [25]:
protein_name = soup.find("div", id="content-protein").find("h1").text

In [26]:
protein_name

'Neuropilin-1'

In [27]:
gene = soup.find("div", id="content-gene").find("h2").text

In [28]:
gene

'NRP1'

In [29]:
function_text = soup.find("div", class_="annotation").find("span").text

In [32]:
text_split = function_text.split("<p>")

In [33]:
text_split[0]

'Receptor involved in the development of the cardiovascular system, in angiogenesis, in the formation of certain neuronal circuits and in organogenesis outside the nervous system. It mediates the chemorepulsant activity of semaphorins (PubMed:9288753, PubMed:9529250, PubMed:10688880). It binds to semaphorin 3A, The PLGF-2 isoform of PGF, The VEGF165 isoform of VEGFA and VEGFB (PubMed:9288753, PubMed:9529250, PubMed:10688880). Coexpression with KDR results in increased VEGF165 binding to KDR as well as increased chemotaxis. Regulates VEGF-induced angiogenesis. Binding to VEGFA initiates a signaling pathway needed for motor neuron axon guidance and cell body migration, including for the caudal migration of facial motor neurons from rhombomere 4 to rhombomere 6 during embryonic development (By similarity). Regulates mitochondrial iron transport via interaction with ABCB8/MITOSUR (PubMed:30623799).By similarity\r\n                                    '